In [1]:
!pip install python-telegram-bot==13.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 495 kB 21.0 MB/s 
     |████████████████████████████████| 423 kB 75.3 MB/s 
     |████████████████████████████████| 58 kB 8.0 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 6.0.4
    Uninstalling tornado-6.0.4:
      Successfully uninstalled tornado-6.0.4
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.2.0
    Uninstalling cachetools-5.2.0:
      Successfully uninstalled cachetools-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=6.0.4, but you have tornado 6.2 which is incompatible.


In [2]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 30.2 MB/s 
     |████████████████████████████████| 1.3 MB 52.2 MB/s 
     |████████████████████████████████| 451 kB 63.0 MB/s 
     |████████████████████████████████| 7.6 MB 62.7 MB/s 
     |████████████████████████████████| 182 kB 65.5 MB/s 
     |████████████████████████████████| 212 kB 71.5 MB/s 
     |████████████████████████████████| 132 kB 51.3 MB/s 
     |████████████████████████████████| 127 kB 82.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=6.0.4, but you have tornado 6.2 which is incompatible.


In [3]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 29.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=394064 sha256=79e60b9b1602de548cb3a68cdc6151f23dd0c24eccd319e2cb512007f2f12729
  Stored in directory: /root/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
Successfully built annoy


In [4]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 8.2 MB 45.7 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=f72c507e855a05034fe6139453d4234871fbb235e3db10c5ec5f0253e749037e
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [5]:
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=62022416f2200a89c7d760f58270f492f31cee4fff060cc8b4f0af4d92e8d95f
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words


In [6]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [7]:
import string
import annoy
import pickle
import numpy as np

from tqdm import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

import re

#### **Обучим модели word2vec, fasttext для вопросно-ответной болталки на основе Multilingual corpus of Questions and Answers (MQA)** (https://huggingface.co/datasets/clips/mqa)

In [8]:
from datasets import load_dataset

cqa_data = load_dataset("clips/mqa", scope="cqa", language="ru")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset mqa downloaded and prepared to /root/.cache/huggingface/datasets/clips___mqa/ru-cqa-question-language=ru,scope=cqa/0.0.0/7eda4cdcbd6f009259fc516f204d776915a5f54ea2ad414c3dcddfaacd4dfe0b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
cqa_data

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'name', 'domain', 'bucket', 'answers'],
        num_rows: 1713712
    })
})

In [64]:
data = cqa_data['train'][:36000]

In [11]:
import pandas as pd

In [65]:
data_df = pd.DataFrame(data, columns=['name', 'answers'])
data_df = data_df.drop(data_df.index[[0,1]]).reset_index(drop=True)
data_df['answers'] = list(map(lambda x: x[0]['text'], data_df['answers']))

In [13]:
data_df.head(5)

,name,answers
0,как начать играть в lineage 2,Гарик игнатов прав. \n\nНо если все еще не п...
1,как через Интернет приобрести комплектующие дл...,"никак , купить новую и всё, а вобще в сервисе ..."
2,Как правильно готовить кольраби?,нареж соломкой и обжарь на оливковом масле с л...
3,Как забыть любимого?,"Для начала - займись собой: маникюр, педикюр,н..."
4,Как стать хакером?,"Методом экстросенсорики, телекинеза и медитации:)"


In [14]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [66]:
question_len = list(map(lambda x: len(x), data_df['name']))
answer_len = list(map(lambda x: len(x), data_df['answers']))

In [67]:
print(" question")
print(" median len = {}\n mean len = {:.2f}\n 0.75 quantile = {}\n 0.85 quantile = {}\n 0.95 quantile = {}\n max = {}\n".format(np.median(question_len), np.mean(question_len), np.quantile(question_len, 0.75), np.quantile(question_len, 0.85), np.quantile(question_len, 0.95), np.max(question_len)))

 question
 median len = 52.0
 mean len = 122.00
 0.75 quantile = 107.0
 0.85 quantile = 161.0
 0.95 quantile = 507.0
 max = 7293



In [68]:
print(" answer")
print(" median len = {}\n mean len = {:.2f}\n 0.75 quantile = {}\n 0.85 quantile = {}\n 0.95 quantile = {}\n max = {}\n".format(np.median(answer_len), np.mean(answer_len), np.quantile(answer_len, 0.75), np.quantile(answer_len, 0.85), np.quantile(answer_len, 0.95), np.max(answer_len)))

 answer
 median len = 222.0
 mean len = 370.94
 0.75 quantile = 368.0
 0.85 quantile = 583.0
 0.95 quantile = 1277.1500000000015
 max = 38620



In [69]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

sentences = list(map(lambda x: preprocess_txt(x), data_df['name'] + ' ' + data_df['answers']))

In [70]:
sentences = [i for i in sentences if len(i) >= 2]

In [71]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=3, workers=8)
modelFT = FastText(sentences=sentences, size=300, min_count=3, window=5, workers=8)

In [72]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

In [73]:
index_map = data_df['answers'].to_dict()

In [74]:
questions = list(map(lambda x: preprocess_txt(x), data_df['name']))

In [75]:
counter = 0

for question in questions:
    n_w2v = 0
    n_ft = 0 
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)   
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

w2v_index.build(10)
ft_index.build(10)

True

In [76]:
def get_response(question, index, model, index_map, count_answer=3):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, count_answer)
    return [index_map[i] for i in answers]

In [77]:
TEXT = "как стать хакером?"

In [78]:
candidate_answer = get_response(TEXT, w2v_index, modelW2V, index_map, count_answer=3)

In [79]:
candidate_answer

['Методом экстросенсорики, телекинеза и медитации:)',
 'представьте такое: вы заходите в школу со сногсшибательным видом. хотите верьте, хотите нет, но никт...',
 'если вы читаете эту страницу, вероятно, вы или подозреваете, или знаете, что вы пассивная личность, ...']

In [80]:
candidate_answer = get_response(TEXT, ft_index, modelFT, index_map, count_answer=3)

In [81]:
candidate_answer

['Методом экстросенсорики, телекинеза и медитации:)',
 'словом «baller» когда-то назвали игроков в мяч, детей улиц, которые перешли в профессиональную игру ...',
 'хотите применять свои великолепные изобразительные таланты и навыки для создания ироничных изображен...']

####**Обучение модели поиска товаров**

In [82]:
shop_data = pd.read_csv("/content/drive/MyDrive/NLP/Course_project/ProductsDataset.csv")


shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data['title_token'] = shop_data['title'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text,title_token
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра...","[юбка, детский, orby]"
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм...",[ботильон]
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер...",[брюки]
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,...","[продать, детский, шапка]"
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли...",[блузка]


In [84]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [85]:
question_data = data_df['name'].to_dict()

In [86]:
idxs = set(np.random.randint(0, len(question_data), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(question_data[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['title_token'].values]

In [87]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [88]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [89]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [90]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.967919025561846

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [92]:
np.mean(tfidf_vect.idf_)

10.565529503496055

In [93]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'туфля': 10.651429991057276,
 'куртка': 7.3555931250529465,
 'девочка': 11.056895099165441,
 'платье': 11.056895099165441,
 'р44': 11.056895099165441,
 'pepe': 11.056895099165441,
 'jeans': 11.056895099165441,
 'кpоватка': 11.056895099165441,
 'bmw': 11.056895099165441,
 'z4': 11.056895099165441,
 'позвонить': 11.056895099165441,
 'домашний': 9.44745718673134,
 'телефон': 10.651429991057276,
 'город': 10.651429991057276,
 'question': 11.056895099165441,
 'сапог': 11.056895099165441,
 'пальто': 11.056895099165441,
 'новый': 11.056895099165441,
 'р4042': 11.056895099165441,
 'комбинезон': 11.056895099165441,
 'женский': 11.056895099165441,
 'жилет': 10.651429991057276,
 'здравствуйте': 9.265135629937385,
 'вопрос': 11.056895099165441,
 'псн': 11.056895099165441,
 'выставлять': 11.056895099165441,
 'счёт': 11.056895099165441,
 'банк': 10.651429991057276,
 'дополнительно': 10.651429991057276,
 'клиент': 11.056895099165441,
 'давать': 11.056895099165441,
 'чек': 11.056895099165441,
 'квита

In [94]:
ft_index_shop = annoy.AnnoyIndex(300 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(300)
    for word in shop_data.loc[i, "title_token"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)

<ipython-input-94-b9c26824bd72>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(shop_data))):


  0%|          | 0/35548 [00:00<?, ?it/s]

<ipython-input-94-b9c26824bd72>:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
<ipython-input-94-b9c26824bd72>:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word] * idfs.get(word, midf)


True

In [47]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(300)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [95]:
ft_index_shop.get_nns_by_vector(np.zeros(300), 1, include_distances=True)

([37], [1.4142135381698608])

#### **Загрузим также предобученную модель для болталки**

In [96]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tinkoff_tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small')
tinkoff_model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [54]:
def tinkoff_gen(inputs):
    generated_token_ids = tinkoff_model.generate(
        **inputs,
        top_k=10,
        top_p=0.95,
        num_beams=3,
        num_return_sequences=3,
        do_sample=True,
        no_repeat_ngram_size=2,
        temperature=1.2,
        repetition_penalty=1.2,
        length_penalty=1.0,
        eos_token_id=50257,
        max_new_tokens=40
    )
    context_with_response = [tinkoff_tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
    cont_resp = context_with_response[np.random.randint(0, len(context_with_response))]
    if re.search('@@ВТОРОЙ@@', cont_resp) != None:
        cont_resp = cont_resp.split('@@ВТОРОЙ@@')[1]
        if re.search('@@ПЕРВЫЙ@@', cont_resp) != None:
            cont_resp = cont_resp.split('@@ПЕРВЫЙ@@')[0]
    cont_resp = re.sub('@@ПЕРВЫЙ@@', ' ', cont_resp)
    
    return cont_resp

####**Запуск бота в Telegram**

In [99]:
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день! Чем могу помочь?')

def echo(update: Update, context: CallbackContext):
  
    txt = update.message.text
    input_txt = preprocess_txt(txt)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 1)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            update.message.reply_text("title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        inputs = tinkoff_tokenizer(txt, return_tensors='pt')
        update.message.reply_text(tinkoff_gen(inputs))
        return
    candidate_answer = get_response(txt, ft_index, modelFT, index_map, count_answer=3)
    update.message.reply_text(candidate_answer[0])

In [100]:
updater = Updater("5853722371:AAH74PJZfRGzbVm4GMH22ruyJ5wsovQonno", use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Start the Bot
updater.start_polling()
updater.idle()

<ipython-input-47-d4b42f9da53d>:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
<ipython-input-47-d4b42f9da53d>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word] * idfs.get(word, midf)
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.
<ipython-input-47-d4b42f9da53d>:8: RuntimeWarning: invalid value encountered in true_divide
  return vec